In [13]:
# Core
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    mean_absolute_error
)

# Models
from lightgbm import LGBMClassifier, LGBMRegressor



Matplotlib is building the font cache; this may take a moment.


In [18]:
df = pd.read_csv("../data/lending_club_80k_clean.csv")
df['emp_length'] = ( df['emp_length'] .astype(str) .str.lower() .str.strip() .str.extract(r'(\d+)') .astype(int) )
df['term'] = ( df['term'] .astype(str) .str.lower() .str.strip() .str.extract(r'(\d+)') .astype(int) )

df["loan_to_income"] = (
    df["loan_amnt"]
    .div(df["annual_inc"])
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .round(3)
)
print(df)
print("Shape:", df.shape)
df.head()


       loan_amnt  annual_inc  fico_score    dti  int_rate  term  emp_length  \
0         7500.0    190000.0       667.0   4.36     16.02    36           3   
1        14000.0     53000.0       677.0  11.96     11.55    36           1   
2         9000.0     70000.0       682.0  17.97     16.78    36          10   
3        31200.0     62400.0       717.0   8.54     26.30    36           2   
4        30000.0     63500.0       787.0   6.44      5.32    36           3   
...          ...         ...         ...    ...       ...   ...         ...   
79995    27975.0     84000.0       777.0  20.21      9.49    60           8   
79996    35000.0     88000.0       682.0  24.34     10.75    60          10   
79997    11000.0     65000.0       722.0   6.68     12.49    36           2   
79998    18600.0     75000.0       672.0  14.16     19.22    36           1   
79999    19000.0    150000.0       737.0  13.06      8.59    36           5   

                  purpose home_ownership  approved 

,loan_amnt,annual_inc,fico_score,dti,int_rate,term,emp_length,purpose,home_ownership,approved,loan_to_income
0,7500.0,190000.0,667.0,4.36,16.02,36,3,debt_consolidation,MORTGAGE,0,0.039
1,14000.0,53000.0,677.0,11.96,11.55,36,1,debt_consolidation,MORTGAGE,1,0.264
2,9000.0,70000.0,682.0,17.97,16.78,36,10,credit_card,RENT,0,0.129
3,31200.0,62400.0,717.0,8.54,26.30,36,2,house,OWN,0,0.500
4,30000.0,63500.0,787.0,6.44,5.32,36,3,debt_consolidation,RENT,1,0.472


In [15]:
df["approved"].value_counts(normalize=True)


approved
1    0.806125
0    0.193875
Name: proportion, dtype: float64